# Getting the rating from the reviews

In [57]:
!pip install -q --disable-pip-version-check -r requirements.txt

In [71]:
# Matrix operations
import numpy as np

# Dataframes
import pandas as pd

# Charts
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# SciKit Learn Utilities
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Horizontal stack in a less memory consuming approach
from scipy.sparse import hstack

# To perform Oversampling
import nlpaug.augmenter.word as naw

# Progress bar
from tqdm import tqdm

# PyTorch Dataset
import torch
from torch.utils.data import DataLoader, Dataset, random_split

# PyTorch Lightning NN Architecture
import pytorch_lightning as pl
import torch.nn.functional as F
from pytorch_lightning import Trainer

## Preprocessing

### 1. Displaying the data

In [29]:
reviews_df = pd.read_csv('Big_AHR.csv', index_col=0)
reviews_df.head()

,title,rating,review_text,location,hotel,label
0,Excelente y personal amable,5,Un hotel muy bueno. El personal fue muy amabl...,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,1
1,Céntrico,4,"Muy buen hotel al nivel de lo esperado, habita...",Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,1
2,Hotel excepcional,5,Magnífico hotel. La verdad es que todo perfect...,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,1
3,WOW!!,5,"Hotel hermoso, buen diseño, original, limpio. ...",Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,1
4,Magnifico,5,Magnífica ubicación en pleno centro de Sevilla...,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,1


Obtain an example from the dataset:

In [30]:
def get_one_sample_with_columns(df, id=0):
    return list(zip(list(df.columns), list(df.iloc[id])))

Display the rating and the sentiment associated to the review:

In [31]:
def display_ratings(df):
    rating = df['rating'].value_counts()

    fig = px.bar(
        x=rating.index,
        y=rating.values,
        labels={'x': 'Rating', 'y': 'Count'},
        title='Count of Each Unique Rating'
    )

    fig.show()

In [32]:
def display_review_sentiment(df):
    SENTIMENT_CONVERSION = { 0: "Negative", 1: "Positive", 3: "Neutral" }

    sentiment_column = df['label'].value_counts().sort_index()

    fig = px.bar(
        # Convert numerical x-axis labels to string labels
        x=[SENTIMENT_CONVERSION[label] for label in sentiment_column.index],
        y=sentiment_column.values,
        labels={'x': 'Sentiment', 'y': 'Count'},
        title='Count of Each Unique Sentiment'
    )

    fig.show()

In [33]:
display_ratings(reviews_df)

In [34]:
display_review_sentiment(reviews_df)

As we can see, the dataset is completely unbalanced. There are almost 10x of 5-star ratings that 2-star ratings.

### 2. Combining the text features

Combining the title and the review_text attributes offer several advantages:

1. Contextual Information: Titles often provide a summary or highlight of the most important aspect of the review. We suspect that combining it with the review text can give the model more context, which might improve its understanding of the review sentiment.

2. Reducing Sparsity: We'll convert the text data into high-dimensional sparse vectors (e.g., via TF-IDF). If each text feature ('title' and 'review_text') is sparse and has little data, combining them can result in a less sparse representation, making it easier for the model to find patterns.

3. Noise Reduction: If one of the text features is noisy or less reliable, combining it with a more informative feature can help "average out" the noise, leading to a more robust representation.

In [35]:
if 'title' in reviews_df.columns.to_list() or 'review_text' in reviews_df.columns.to_list():
    reviews_df['text_content'] = reviews_df['title'] + ' ' + reviews_df['review_text']
    reviews_df = reviews_df.drop(columns=['title', 'review_text'])

Reorder the columns just to improve its readability:

In [36]:
new_column_order = ['location', 'hotel', 'text_content', 'label', 'rating']
reviews_df = reviews_df[new_column_order]

In [37]:
reviews_df.head()

,location,hotel,text_content,label,rating
0,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,Excelente y personal amable Un hotel muy bueno...,1,5
1,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,Céntrico Muy buen hotel al nivel de lo esperad...,1,4
2,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,Hotel excepcional Magnífico hotel. La verdad e...,1,5
3,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,"WOW!! Hotel hermoso, buen diseño, original, li...",1,5
4,Seville_Province_of_Seville_Andalucia,H10_Casa_de_la_Plata,Magnifico Magnífica ubicación en pleno centro ...,1,5


### 3. Exploring the data

In [38]:
def has_nan_or_missing_values(df):
    # Check for NaN values
    nan_check = df.isna().any()

    # Check for empty values (assuming empty strings)
    empty_check = (df == '').any()

    # Combine the results to check for both NaN and empty values
    has_nan_or_empty = nan_check | empty_check

    print(has_nan_or_empty)
    print(f"Number of samples: {df.shape[0]}")

In [39]:
has_nan_or_missing_values(reviews_df)

location         True
hotel            True
text_content    False
label           False
rating          False
dtype: bool
Number of samples: 18172


In [40]:
pd.unique(reviews_df['location'])[-5:]

array(['anlucar_de_Barrameda_Province_of_Cadiz_Andalucia',
       'Alajar_Sierra_de_Aracena_and_Picos_de_Aroche_Natural_Park_Pro',
       'Alajar_Sierra_de_Aracena_and_Picos_de_Aroche_Natural_Park_Province_of',
       'Tavira_Faro_District_Algarve', nan], dtype=object)

As we can see, there's a NaN so we'll remove it:

In [41]:
reviews_df = reviews_df.dropna(subset=['location'])

In [42]:
has_nan_or_missing_values(reviews_df)

location        False
hotel           False
text_content    False
label           False
rating          False
dtype: bool
Number of samples: 16356


✅ Removing the NaN values resulted on also removing the NaN and missing values from the hotel attribute.

In [43]:
get_one_sample_with_columns(reviews_df)

[('location', 'Seville_Province_of_Seville_Andalucia'),
 ('hotel', 'H10_Casa_de_la_Plata'),
 ('text_content',
  'Excelente y personal amable Un hotel muy bueno.  El personal fue muy amable y profesional. Nos gustaban desayuno mucho también. El habitación cómoda y limpia. Volvimos a próxima vez en Seville por seguro. Lo recomiendo !! Y también ubicación muy cerca del centro :))))'),
 ('label', 1),
 ('rating', 5)]

Obtain more information about the dataframe:

In [44]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16356 entries, 0 to 16355
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   location      16356 non-null  object
 1   hotel         16356 non-null  object
 2   text_content  16356 non-null  object
 3   label         16356 non-null  int64 
 4   rating        16356 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 766.7+ KB


Just for efficiency purposes, we can change the type of the label and rating columns:

In [45]:
reviews_df = reviews_df.astype({'label': 'uint8', 'rating': 'uint8'})

In [46]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16356 entries, 0 to 16355
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   location      16356 non-null  object
 1   hotel         16356 non-null  object
 2   text_content  16356 non-null  object
 3   label         16356 non-null  uint8 
 4   rating        16356 non-null  uint8 
dtypes: object(3), uint8(2)
memory usage: 543.1+ KB


### 4. Balance the data

We'll perform text oversampling in order to avoid overfitting while balancing the dataset.

For that, we'll use the library nlpaug which will allow us to perform text augmentation by using synonyms.

In [ ]:
aug = naw.SynonymAug(aug_src='wordnet', lang='spa', aug_max=100)

def augment_text(text):
    return aug.augment(text)[0]

In [ ]:
def oversample(reviews_df, target_samples):
    new_reviews = []

    for rating, target_count in tqdm(target_samples.items(), desc='Oversampling Ratings', unit='rating', total=len(target_samples)):
        # Filter reviews with the current rating
        ratings = reviews_df[reviews_df['rating'] == rating] 

        if target_count > len(ratings):
            # Number of samples to generate
            oversample_count = target_count - len(ratings)
            
            for _ in range(oversample_count):
                # Randomly select a review to augment
                row = ratings.sample(1).iloc[0]
                
                augmented_text = augment_text(row['text_content'])
                
                new_reviews.append({'location': row['location'],
                                    'hotel': row['hotel'],
                                    'text_content': augmented_text,
                                    'label': row['label'],
                                    'rating': rating})

    return pd.DataFrame(new_reviews)

In [ ]:
max_rating_samples = max([len(reviews_df[reviews_df['rating'] == rating]) for rating in [1,2,3,4,5]])
print(f"Maximum number of samples in a rating: {max_rating_samples}")

Maximum number of samples in a rating: 8474


In [ ]:
target_samples = {1: max_rating_samples, 2: max_rating_samples, 3: max_rating_samples, 4: max_rating_samples, 5: max_rating_samples}
new_df = oversample(reviews_df, target_samples)

Oversampling Ratings: 100%|██████████| 5/5 [02:31<00:00, 30.28s/rating]


In [ ]:
new_df

,location,hotel,text_content,label,rating
0,Malaga_Costa_del_Sol_Province_of_Malaga_Andalucia,Petit_Malaga_Congress,TERRIBLE HABITACION SIN VENTANA Copio y pego e...,0,1
1,Almeria_Province_of_Almeria_Andalucia,Hostal_Estacion,Peor sitio primer estado mi existencia He leíd...,0,1
2,Almeria_Province_of_Almeria_Andalucia,La_Gallineta_de_Gata,"Bochornoso, lamentable! 0% recomendable, razon...",0,1
3,Seville_Province_of_Seville_Andalucia,Simon_Hotel,La peor dormitorio que he visto nunca Salgo de...,0,1
4,Cordoba_Province_of_Cordoba_Andalucia,Cordoba_Inturjoven_Youth_Hostel,"Horrible masía Había estado hacía años, y tení...",0,1
...,...,...,...,...,...
26009,Cordoba_Province_of_Cordoba_Andalucia,Eurostars_Maimonides,Muy bien situado La localización perfecta a es...,1,4
26010,Cordoba_Province_of_Cordoba_Andalucia,Exe_Ciudad_de_Cordoba,Buena ubicación Aunque no se encuentra céntric...,1,4
26011,Granada_Province_of_Granada_Andalucia,Palacio_Almagra,Excelente Hemos pasado una atardecer en el dom...,1,4
26012,Seville_Province_of_Seville_Andalucia,Ribera_de_Triana_Hotel,Nox de bodas Pasamos una noche en Sevilla en e...,1,4


Append augmented data to the original dataframe

In [ ]:
reviews_df = pd.concat([reviews_df, new_df], axis=0)

Shuffle the data:

In [ ]:
reviews_df = shuffle(reviews_df).reset_index(drop=True)

As we can see, now the dataset are more balanced:

In [ ]:
display_ratings(reviews_df)

In [ ]:
display_review_sentiment(reviews_df)

### 5. Save balanced dataset

In [ ]:
reviews_df.to_csv('Big_AHR_Balanced.csv')

If the reviews_df is not defined (in case we jump directly to point 5), import it:

In [ ]:
if not 'reviews_df' in locals():
    reviews_df = pd.read_csv('Big_AHR_Balanced.csv', index_col=0)

### 6. Convert categorical data into numerical format using One-Hot Encoding

We have to convert the location and hotel to numerical format using One-Hot Encoding:

In [47]:
encoder = OneHotEncoder(sparse=False)
categorical_features = encoder.fit_transform(reviews_df[['location', 'hotel']])

### 7. Convert the text data into numerical format using TF-IDF

We've to convert the text data (which is the attribute text_content) into a numerical format so it could be used by the neural network. In order to do this, we'll use TF-IDF (Term Frequency - Inverse Document Frequency), which is a technique to quantify words in a set of documents:

In [48]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
text_features = tfidf_vectorizer.fit_transform(reviews_df['text_content'])

In [49]:
print(f"Number of samples: {len(text_features.toarray())}")

Number of samples: 16356


### 8. Join the one-hot-encoded data and the text in numerical format

In [50]:
combined_features = hstack([categorical_features, text_features]).tocsr()

### 9. Visualize the features using T-SNE by applying Dimensionality Reduction (Optional)

In [ ]:
# Dimensionality reduction to improve the visualization
# - We're reducing to 50 components as an example, but this can be changed
svd = TruncatedSVD(n_components=50)
reduced_features = svd.fit_transform(combined_features)

Apply T-SNE (this lasts like 5min):

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(reduced_features)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
# Create a DataFrame for the t-SNE results
tsne_df = pd.DataFrame(data={
    'X': tsne_results[:, 0],
    'Y': tsne_results[:, 1],
    'Rating': reviews_df['rating'],
    'Label': reviews_df['label']
})

# Plot using Plotly with Rating as color and Label as marker style
fig = px.scatter(tsne_df, x='X', y='Y', color='Rating', symbol='Label', 
                 color_continuous_scale='viridis', 
                 title="t-SNE Visualization of Reviews",
                 symbol_map={0: 'circle', 1: 'square', 3: 'cross'}
                )

# Adjust the order of the legends
fig.update_layout(coloraxis_colorbar=dict(title='Rating'))
fig.update_xaxes(title_text='t-SNE Component 1')
fig.update_yaxes(title_text='t-SNE Component 2')
fig.show()


### 10. Split the data

Convert the join features into just one dataframe:

In [51]:
# text_content, hotel, location
curated_reviews_df = pd.DataFrame(combined_features.toarray())

Add the label attribute:

In [52]:
# label
curated_reviews_df['label'] = reviews_df['label']

Split the data into train and test:

In [53]:
X = curated_reviews_df
y = reviews_df['rating']

In [59]:
# Split the data into train, validation, and test set (e.g., 80%, 10%, 10%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=1/9, random_state=42)

As we can see, now we have:

- Training: X_train, y_train.

- Validation: X_val, y_val.

- Testing: X_test, y_test.

## Model

In [63]:
BATCH_SIZE = 32
NUM_EPOCHS = 10

### 1. Define a PyTorch dataset and dataloader

In [60]:
class ReviewsDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [62]:
train_dataset = ReviewsDataset(torch.tensor(X_train.values).float(), torch.tensor(y_train.values).long())
val_dataset = ReviewsDataset(torch.tensor(X_val.values).float(), torch.tensor(y_val.values).long())
test_dataset = ReviewsDataset(torch.tensor(X_test.values).float(), torch.tensor(y_test.values).long())

In [64]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

### 2. Neural Network Architecture using PyTorch Lightning

In [72]:
class ReviewClassifier(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super(ReviewClassifier, self).__init__()
        # Fully Connected Layers (input_dim, 256, 128, output_dim)
        self.fc1 = torch.nn.Linear(input_dim, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, output_dim)

        # Trackers for our desired metrics
        self.train_losses = []
        self.val_losses = []
        self.val_accs = []
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def training_step(self, batch, batch_idx):
        # Obtaining the x and y value of a batch from the dataset with size BATCH_SIZE
        x, y = batch
        # Predict the value of y using x
        y_hat = self(x)

        # Loss Function: We'll use the Cross-Entropy Loss as it's a loss function used for multi-class
        # classification problems (thanks to using the softmax activation function)
        loss = F.cross_entropy(y_hat, y)
        
        return loss

    def training_epoch_end(self, outputs):
        # Compute the average training loss over the epoch and add it to the list of losses
        avg_train_loss = torch.stack(x['loss'] for x in outputs).mean()
        self.train_losses.append(avg_train_loss)
    
    def validation_epoch_end(self, outputs):
        avg_val_loss = torch.stack(x['loss'] for x in outputs).mean()
        avg_acc = sum([x["val_acc"] for x in outputs]) / len(outputs)
        self.val_losses.append(avg_val_loss)
        self.val_accs.append(avg_acc)
        metrics = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return {'val_loss': avg_loss, 'progress_bar': metrics}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y)

        # We'll use the accuracy as the metric for the classification problem
        # - pred_val = torch.argmax in dim=1 (y-axis): [[0.1, 0.8, 0.1], 
        #                                               [0.7, 0.1, 0.2]] -> output: [1, 0]
        # - v = torch.sum(pred_val == [1, 1]) -> output: sum([True, False]) -> output: 1 (.item() gets the val)
        # - v / [1, 1].shape[0] -> output: 1 / 2 -> 50% of accuracy
        val_acc = torch.sum(torch.argmax(y_hat, dim=1) == y).item() / y.shape[0]

        return {'val_loss': avg_loss, 'progress_bar': metrics}
    
    def configure_optimizers(self):
        # We'll use Adam as the optimizer as it has shown pretty good results in a lot of
        # tasks where classification is performed
        return torch.optim.Adam(self.parameters(), lr=0.001) 

In [73]:
input_dim = X_train.shape[1]
output_dim = y_train.nunique()

print(f"Input dimension: {input_dim} (input neurons)\nOutput dimension: {output_dim} (num of different ratings)")

Input dimension: 5749 (input neurons)
Output dimension: 5 (num of different ratings)


In [74]:
model = ReviewClassifier(input_dim, output_dim)

### 3. Training the model

In [ ]:
trainer = Trainer(max_epochs=NUM_EPOCHS, progress_bar_refresh_rate=20)
trainer.fit(model, train_loader, val_loader)

## Results

### 1. Visualizing the Loss and Accuracy over the epochs

In [ ]:
# Extract values from the model
train_losses = model.train_losses
val_losses = model.val_losses
val_accs = model.val_accs

In [ ]:
# Create the plots
fig = go.Figure()

# Loss plots
fig.add_trace(go.Scatter(y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(y=val_losses, mode='lines', name='Validation Loss'))

# Accuracy plot
fig.add_trace(go.Scatter(y=val_accs, mode='lines', name='Validation Accuracy'))

# Layout options
fig.update_layout(
    title='Training Metrics over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Value'
)

fig.show()

### 2. Confussion matrix

In [ ]:
model.eval()
all_preds = []
all_labels = []

In [ ]:
with torch.no_grad():
    for batch in test_loader:
        # Get a batch, predict the rating and save the prediction and the true value
        x, y = batch
        y_hat = model(x)
        all_preds.extend(torch.argmax(y_hat, dim=1).numpy())
        all_labels.extend(y.numpy())

Show the confusion matrix with the computed predictions and true labels:

In [ ]:
conf_matrix = confusion_matrix(all_labels, all_preds)

Plot the results:

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix, annot=True, fmt="d")
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cc825a7d-bfd6-4172-a68d-91ec53531f43' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>